# Particle Tracking Tutorial using Trackpy

This notebook provides a simple, bare-bones workflow for particle tracking analysis using the trackpy library.

## Overview
- **Cell 0**: Setup and imports
- **Cell 1**: Load TIF file and display frame count
- **Cell 2**: Display a frame and intensity histogram
- **Cell 3**: Locate particles in a single frame
- **Cell 4**: Batch process multiple frames
- **Cell 5**: Link trajectories across frames

## Reference
This tutorial follows the trackpy walkthrough: https://soft-matter.github.io/trackpy/v0.7/tutorial/walkthrough.html

## Cell 0: Setup & Imports

Import all required libraries.

In [ ]:
import trackpy as tp
import pims
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog

print("✓ All libraries imported successfully")
print(f"Trackpy version: {tp.__version__}")

## Cell 1: Browse and Load TIF File

Use a file dialog to select and load a TIF file. The file is loaded using pims for memory-efficient lazy loading.

In [ ]:
# Browse for file
root = Tk()
root.withdraw()
filepath = filedialog.askopenfilename(filetypes=[("TIF files", "*.tif *.tiff")])
root.destroy()

# Load frames
frames = pims.open(filepath)
print(f"Loaded {len(frames)} frames from: {filepath}")

## Cell 2: Display a Frame and Intensity Histogram

Select a frame to display and view its intensity distribution. Adjust `frame_num` to view different frames. Optionally set `min_threshold` and `max_threshold` for visualization.

In [ ]:
# Parameters - modify these as needed
frame_num = 0
min_threshold = None  # Set to a value if you want to apply a minimum threshold
max_threshold = None  # Set to a value if you want to apply a maximum threshold

# Get the frame
frame = frames[frame_num]

# Create side-by-side plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Display frame
im = ax1.imshow(frame, cmap='gray', vmin=min_threshold, vmax=max_threshold)
ax1.set_title(f'Frame {frame_num}')
ax1.axis('off')
plt.colorbar(im, ax=ax1)

# Display intensity histogram
ax2.hist(frame.flatten(), bins=100, color='blue', alpha=0.7)
ax2.set_xlabel('Intensity')
ax2.set_ylabel('Frequency')
ax2.set_title('Intensity Histogram')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Frame shape: {frame.shape}")
print(f"Intensity range: [{frame.min()}, {frame.max()}]")

## Cell 3: Locate Particles in Single Frame

Use trackpy's `locate()` function to find particles in a single frame. Adjust `diameter` and `minmass` parameters to optimize particle detection.

In [ ]:
# Parameters - modify these as needed
frame_num = 0
diameter = 11  # Feature size in pixels (must be odd)
minmass = 1000  # Minimum integrated brightness

# Locate particles
frame = frames[frame_num]
f = tp.locate(frame, diameter=diameter, minmass=minmass)

print(f"Found {len(f)} particles in frame {frame_num}")
print(f"\nParticle data (first 5 rows):")
print(f.head())

# Create visualization with 3 subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Annotated image
ax1 = axes[0, 0]
ax1.imshow(frame, cmap='gray')
tp.annotate(f, frame, ax=ax1)
ax1.set_title(f'Detected Particles: {len(f)}')
ax1.axis('off')

# 2. Particle mass histogram
ax2 = axes[0, 1]
ax2.hist(f['mass'], bins=30, color='green', alpha=0.7)
ax2.set_xlabel('Mass')
ax2.set_ylabel('Count')
ax2.set_title('Particle Mass Distribution')
ax2.grid(True, alpha=0.3)

# 3. Subpixel bias diagnostic
ax3 = axes[1, 0]
tp.subpx_bias(f, ax=ax3)
ax3.set_title('Subpixel Bias')

# 4. Size vs Mass
ax4 = axes[1, 1]
ax4.scatter(f['size'], f['mass'], alpha=0.5)
ax4.set_xlabel('Size')
ax4.set_ylabel('Mass')
ax4.set_title('Size vs Mass')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Cell 4: Batch Process Frames

Process multiple frames using trackpy's `batch()` function. Set the frame range and use the same parameters from Cell 3.

In [ ]:
# Parameters - modify these as needed
start_frame = 0
end_frame = 50  # Process frames 0-50
diameter = 11
minmass = 1000

# Batch process frames
print(f"Processing frames {start_frame} to {end_frame}...")
# Note: Python slicing handles end_frame+1 gracefully if end_frame >= len(frames)
f_batch = tp.batch(frames[start_frame:end_frame+1], diameter=diameter, minmass=minmass)

print(f"\nTotal particles detected: {len(f_batch)}")
print(f"Frames processed: {f_batch['frame'].nunique()}")
print(f"\nBatch data (first 5 rows):")
print(f_batch.head())

# Calculate particles per frame
particles_per_frame = f_batch.groupby('frame').size()

# Plot particles per frame
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 1. Particle count per frame
ax1.plot(particles_per_frame.index, particles_per_frame.values, 'o-', alpha=0.7)
ax1.set_xlabel('Frame Number')
ax1.set_ylabel('Particle Count')
ax1.set_title('Particles Detected per Frame')
ax1.grid(True, alpha=0.3)

# 2. Distribution of particle counts
ax2.hist(particles_per_frame.values, bins=20, color='purple', alpha=0.7)
ax2.set_xlabel('Particles per Frame')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Particle Counts')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAverage particles per frame: {particles_per_frame.mean():.1f}")
print(f"Min particles: {particles_per_frame.min()}")
print(f"Max particles: {particles_per_frame.max()}")

## Cell 5: Link Trajectories

Link particles across frames into trajectories using trackpy's `link()` function. Set `search_range` (maximum distance particles can move between frames) and `memory` (number of frames a particle can disappear and reappear).

In [ ]:
# Parameters - modify these as needed
search_range = 5  # Maximum distance particles can move between frames
memory = 3  # Number of frames a particle can vanish and reappear
min_track_length = 10  # Minimum number of frames for a valid trajectory

# Link particles into trajectories
print("Linking trajectories...")
t = tp.link(f_batch, search_range=search_range, memory=memory)

print(f"\nTotal trajectories found: {t['particle'].nunique()}")
print(f"\nTrajectory data (first 5 rows):")
print(t.head())

# Filter short trajectories
t_filtered = tp.filter_stubs(t, threshold=min_track_length)
print(f"\nTrajectories after filtering (>= {min_track_length} frames): {t_filtered['particle'].nunique()}")

# Calculate track lengths
track_lengths = t_filtered.groupby('particle').size()

# Create visualization with 3 subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Track length histogram
ax1 = axes[0, 0]
ax1.hist(track_lengths, bins=30, color='orange', alpha=0.7)
ax1.set_xlabel('Track Length (frames)')
ax1.set_ylabel('Count')
ax1.set_title('Track Length Distribution')
ax1.grid(True, alpha=0.3)

# 2. Plot trajectories using trackpy's built-in function
ax2 = axes[0, 1]
tp.plot_traj(t_filtered, ax=ax2)
ax2.set_title(f'Particle Trajectories (n={t_filtered["particle"].nunique()})')

# 3. Particle positions colored by track
ax3 = axes[1, 0]
for particle_id in t_filtered['particle'].unique()[:50]:  # Plot first 50 tracks for clarity
    track = t_filtered[t_filtered['particle'] == particle_id]
    ax3.plot(track['x'], track['y'], 'o-', alpha=0.5, markersize=2)
ax3.set_xlabel('x (pixels)')
ax3.set_ylabel('y (pixels)')
ax3.set_title('Particle Tracks (first 50)')
ax3.invert_yaxis()
ax3.grid(True, alpha=0.3)

# 4. Trajectory statistics
ax4 = axes[1, 1]
ax4.axis('off')
stats_text = f"""
Trajectory Statistics:
━━━━━━━━━━━━━━━━━━━━━━
Total trajectories: {t['particle'].nunique()}
Filtered trajectories: {t_filtered['particle'].nunique()}

Track Length:
  Mean: {track_lengths.mean():.1f} frames
  Median: {track_lengths.median():.1f} frames
  Min: {track_lengths.min()} frames
  Max: {track_lengths.max()} frames

Parameters Used:
  search_range: {search_range}
  memory: {memory}
  min_track_length: {min_track_length}
"""
ax4.text(0.1, 0.5, stats_text, fontsize=11, family='monospace', verticalalignment='center')

plt.tight_layout()
plt.show()

# Display sample trajectory data
print("\nSample trajectory (particle 0):")
print(t_filtered[t_filtered['particle'] == t_filtered['particle'].unique()[0]].head(10))